In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import time
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [3]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
job=importlib.reload(job)
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
importlib.reload(orderbook.ordertree)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [4]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50
df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})

In [5]:
data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2) ##Careful - converts to int32 in jnp array, meaning the filler values get truncated.
newarr=jnp.zeros((int(orderbookLevels*2),8))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200).at[:,7].set(0).astype('int32')



df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]
test=df_orders["Time"].astype('string').str.split('.',expand=True)
df_orders[["TimeWhole","TimeDec"]]=test.astype('int32')#.rename(['TimeWhole','TimeDec'])
df_orders=df_orders.drop("Time",axis=1)
ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 8)


In [6]:
#horizon=13519
horizon=16643
#horizon=87
orderbook_1=JaxOrderBook(price_levels=150,orderQueueLen=100)
trades_init=orderbook_1.process_orders_array(initOB)
t1=time.time()
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon]).block_until_ready()
t2=time.time()
#trades_msgs=orderbook_1.process_orders_array(ordersJNP[horizon-1:horizon])
print(t2-t1)

6.037456274032593


In [7]:
MINPRICE=-1.410065407e9
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1])
L2OB=L2OB.at[L2OB==MINPRICE].set(-1)


'''
print(ordersJNP[horizon])
print('##################   ORDERBOOK DELTA   ##########################')
print((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB
print('############################################')
print(L2OB.reshape(50,4))
print('############################################')
print(myOB.reshape(100,4)[0:60,:])
'''




"\nprint(ordersJNP[horizon])\nprint('##################   ORDERBOOK DELTA   ##########################')\nprint((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB\nprint('############################################')\nprint(L2OB.reshape(50,4))\nprint('############################################')\nprint(myOB.reshape(100,4)[0:60,:])\n"

## Load the original orderbook implementation on the CPU

In [405]:
def type_side_converter(type : int, side : int):
    side_ret = 'bid' if side ==1 else 'ask'
    if type==1:
        return 'limit',side_ret
    elif type==2:
        return 'cancel',side_ret
    elif type==3:
        return 'delete',side_ret
    elif type==4:
        side_ret = 'ask' if side ==1 else 'bid'
        return 'limit',side_ret
    else:
        return 'skip',side_ret

In [406]:
orderbook_2=cpuOrderBook()
limit_orders = []
INITID=1500000
order_id_list = [orderbook.orderbook.INITID + i for i in range(100)]
for i in range(50):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
for i in range(50):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

print(orderbook_2)

====================================== Asks ======================================
Quantity    30  |  Price  2900000  |  Order_ID   90000049  |  Time 34200.000000001
Quantity    16  |  Price  2887300  |  Order_ID   90000048  |  Time 34200.000000001
Quantity    15  |  Price  2850000  |  Order_ID   90000047  |  Time 34200.000000001
Quantity    10  |  Price  2796200  |  Order_ID   90000046  |  Time 34200.000000001
Quantity    50  |  Price  2780000  |  Order_ID   90000045  |  Time 34200.000000001
Quantity    20  |  Price  2750000  |  Order_ID   90000044  |  Time 34200.000000001
Quantity   100  |  Price  2675100  |  Order_ID   90000043  |  Time 34200.000000001
Quantity    10  |  Price  2675000  |  Order_ID   90000042  |  Time 34200.000000001
Quantity   100  |  Price  2674900  |  Order_ID   90000041  |  Time 34200.000000001
Quantity   100  |  Price  2674600  |  Order_ID   90000040  |  Time 34200.000000001
Quantity    76  |  Price  2656200  |  Order_ID   90000039  |  Time 34200.000000001
Quan

In [407]:
##Taken largely from the Example.py file in the orderbook repository for the cpu
t1=time.time()
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype,side=type_side_converter(l1[1],l1[5])
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    print(index,message)
    orderbook_2.processOrder(message,True,False)

t2=time.time()
        
print("The Final order is:")
print(message)

0 {'type': 'skip', 'side': 'bid', 'quantity': 13, 'price': 2228300, 'trade_id': 0, 'timestamp': '34200.00678936', 'order_id': 0}
1 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 1870200, 'trade_id': 7381730, 'timestamp': '34200.030569044', 'order_id': 7381730}
2 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2585700, 'trade_id': 7381891, 'timestamp': '34200.030920932', 'order_id': 7381891}
3 {'type': 'skip', 'side': 'ask', 'quantity': 17011, 'price': 2228700, 'trade_id': -1, 'timestamp': '34200.034289918', 'order_id': -1}
4 {'type': 'limit', 'side': 'bid', 'quantity': 11, 'price': 2227500, 'trade_id': 5519048, 'timestamp': '34200.034289918', 'order_id': 5519048}
5 {'type': 'limit', 'side': 'ask', 'quantity': 10, 'price': 2230000, 'trade_id': 2142274, 'timestamp': '34200.034289918', 'order_id': 2142274}
6 {'type': 'limit', 'side': 'bid', 'quantity': 200, 'price': 2226000, 'trade_id': 2629509, 'timestamp': '34200.034289918', 'order_id': 2629509}
7 {'type': 'limit'

2240 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2219600, 'trade_id': 15883866, 'timestamp': '34769.920948472', 'order_id': 15883866}
should not be here
Quantity   100  |  Price  2219600  |  Order_ID   15883866  |  Time 34769.395765432
2241 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2219300, 'trade_id': 15905724, 'timestamp': '34769.920948472', 'order_id': 15905724}
2242 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2240400, 'trade_id': 15905729, 'timestamp': '34769.921027787', 'order_id': 15905729}
2243 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2239500, 'trade_id': 15883490, 'timestamp': '34769.921289155', 'order_id': 15883490}
2244 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2229200, 'trade_id': 15905750, 'timestamp': '34769.921558208', 'order_id': 15905750}
2245 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2225400, 'trade_id': 15905770, 'timestamp': '34769.922395595', 'order_id': 15905770}
22

3433 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2216800, 'trade_id': 19638916, 'timestamp': '35051.649688464', 'order_id': 19638916}
3434 {'type': 'delete', 'side': 'bid', 'quantity': 200, 'price': 2208000, 'trade_id': 19318388, 'timestamp': '35051.793199801', 'order_id': 19318388}
should not be here
Quantity   200  |  Price  2208000  |  Order_ID   19318388  |  Time 35031.79323193 
3435 {'type': 'delete', 'side': 'ask', 'quantity': 300, 'price': 2215000, 'trade_id': 19617151, 'timestamp': '35051.894558239', 'order_id': 19617151}
3436 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2216800, 'trade_id': 19638916, 'timestamp': '35051.894605069', 'order_id': 19638916}
3437 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2215200, 'trade_id': 19642201, 'timestamp': '35051.894605069', 'order_id': 19642201}
3438 {'type': 'delete', 'side': 'ask', 'quantity': 200, 'price': 2215100, 'trade_id': 19617143, 'timestamp': '35051.897210559', 'order_id': 19617143}


Quantity   100  |  Price  2211200  |  Order_ID   19858761  |  Time 35068.85383308 
3533 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2212300, 'trade_id': 19859489, 'timestamp': '35068.919196449', 'order_id': 19859489}
3534 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2211800, 'trade_id': 19859239, 'timestamp': '35068.919273937', 'order_id': 19859239}
should not be here
Quantity   100  |  Price  2211800  |  Order_ID   19859239  |  Time 35068.900932964
3535 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2212300, 'trade_id': 19859489, 'timestamp': '35068.921720425', 'order_id': 19859489}
should not be here
Quantity   100  |  Price  2212300  |  Order_ID   19859489  |  Time 35068.919196449
3536 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2211800, 'trade_id': 19859560, 'timestamp': '35068.921868855', 'order_id': 19859560}
3537 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2212400, 'trade_id': 19859584, 'timestamp': '35068.

5860 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2182000, 'trade_id': 23975522, 'timestamp': '35348.542041076', 'order_id': 23975522}
should not be here
Quantity   100  |  Price  2182000  |  Order_ID   23975522  |  Time 35347.846592689
5861 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2184800, 'trade_id': 23986737, 'timestamp': '35348.542205971', 'order_id': 23986737}
5862 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2184800, 'trade_id': 23986737, 'timestamp': '35348.542305892', 'order_id': 23986737}
should not be here
Quantity   100  |  Price  2184800  |  Order_ID   23986737  |  Time 35348.542205971
5863 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2197800, 'trade_id': 23983514, 'timestamp': '35348.542417284', 'order_id': 23983514}
should not be here
Quantity   100  |  Price  2197800  |  Order_ID   23983514  |  Time 35348.403937475
5864 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2182500, 'trade_id': 23986754, 

...

11741 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2188400, 'trade_id': 32680400, 'timestamp': '36078.398853516', 'order_id': 32680400}
11742 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2188900, 'trade_id': 32984352, 'timestamp': '36078.398970273', 'order_id': 32984352}
11743 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2196800, 'trade_id': 32674953, 'timestamp': '36079.70453327', 'order_id': 32674953}
11744 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2191900, 'trade_id': 33015099, 'timestamp': '36079.704771525', 'order_id': 33015099}
11745 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2186200, 'trade_id': 32932718, 'timestamp': '36079.95384516', 'order_id': 32932718}
should not be here
Quantity   100  |  Price  2186200  |  Order_ID   32932718  |  Time 36076.614924593
11746 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2186500, 'trade_id': 33019315, 'timestamp': '36079.953931689', 'order_id': 3301931

12739 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2178100, 'trade_id': 33929865, 'timestamp': '36123.754129492', 'order_id': 33929865}
12740 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2178100, 'trade_id': 33929865, 'timestamp': '36123.756584674', 'order_id': 33929865}
12741 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2178000, 'trade_id': 33929913, 'timestamp': '36123.756661481', 'order_id': 33929913}
12742 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2178000, 'trade_id': 33929913, 'timestamp': '36123.758999654', 'order_id': 33929913}
12743 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2177900, 'trade_id': 33929949, 'timestamp': '36123.75907452', 'order_id': 33929949}
12744 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2177900, 'trade_id': 33929949, 'timestamp': '36123.761513786', 'order_id': 33929949}
12745 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2177800, 'trade_id': 33929986, 't

13721 {'type': 'skip', 'side': 'bid', 'quantity': 100, 'price': 2168700, 'trade_id': 0, 'timestamp': '36181.429502013', 'order_id': 0}
13722 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2166400, 'trade_id': 35080018, 'timestamp': '36181.43164358', 'order_id': 35080018}
13723 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2166300, 'trade_id': 35077517, 'timestamp': '36181.431710535', 'order_id': 35077517}
should not be here
Quantity   100  |  Price  2166300  |  Order_ID   35077517  |  Time 36181.234782176
13724 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2166500, 'trade_id': 35080026, 'timestamp': '36181.431711221', 'order_id': 35080026}
13725 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2151400, 'trade_id': 35057306, 'timestamp': '36181.431856852', 'order_id': 35057306}
should not be here
Quantity   100  |  Price  2151400  |  Order_ID   35057306  |  Time 36179.095482652
13726 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'pri

15119 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2173800, 'trade_id': 37168927, 'timestamp': '36285.491811231', 'order_id': 37168927}
should not be here
Quantity   100  |  Price  2173800  |  Order_ID   37168927  |  Time 36284.76792294 
15120 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2173900, 'trade_id': 37180706, 'timestamp': '36285.49191772', 'order_id': 37180706}
15121 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2173900, 'trade_id': 37180706, 'timestamp': '36285.585840221', 'order_id': 37180706}
should not be here
Quantity   100  |  Price  2173900  |  Order_ID   37180706  |  Time 36285.49191772 
15122 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2174000, 'trade_id': 37186580, 'timestamp': '36285.585925088', 'order_id': 37186580}
15123 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2176100, 'trade_id': 37177943, 'timestamp': '36285.585994088', 'order_id': 37177943}
15124 {'type': 'limit', 'side': 'ask', 'quan

16098 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2170100, 'trade_id': 38082839, 'timestamp': '36349.550337439', 'order_id': 38082839}
16099 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2169900, 'trade_id': 38082922, 'timestamp': '36349.552569723', 'order_id': 38082922}
16100 {'type': 'limit', 'side': 'ask', 'quantity': 27, 'price': 2170100, 'trade_id': 38086414, 'timestamp': '36349.655993717', 'order_id': 38086414}
16101 {'type': 'delete', 'side': 'ask', 'quantity': 27, 'price': 2170100, 'trade_id': 38086414, 'timestamp': '36349.675450752', 'order_id': 38086414}
16102 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2170200, 'trade_id': 38080987, 'timestamp': '36350.353801306', 'order_id': 38080987}
16103 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2170000, 'trade_id': 38098037, 'timestamp': '36350.353801306', 'order_id': 38098037}
16104 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2177700, 'trade_id': 37948370, 'ti

In [408]:
ordersJNP[horizon] #25590397

Array([        3,         1,        20,   2160000,         0,  26671093,
           36395, 537433168], dtype=int32)

In [409]:
t2-t1

1.0718920230865479

In [410]:
from itertools import zip_longest
cpuOB=jnp.array(list(zip_longest(*orderbook_2.get_L2_state(), fillvalue=-1)))

print('#'*15+" Difference between the JAX (GPU) and Python (CPU) orderbook implementations"+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print((cpuOB[:50,:]-myOB.reshape(150,4)[0:50,:]))
print('#'*15+" Python (CPU) orderbook "+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print(cpuOB[:50,:])
print('#'*15+" JAX (GPU) orderbook "+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print(myOB.reshape(150,4)[0:50,:])

AttributeError: 'OrderBook' object has no attribute 'get_L2_state'

In [382]:
print(orderbook_2.bids.get_price_list(2160000))

Quantity    29  |  Price  2160000  |  Order_ID   38916890  |  Time 36391.009541395



In [ ]:
print(orderbook_2.bids.order_map[26671093])

In [ ]:
orderbook_1.orderbook_array[1,0,:,:]

In [ ]:
testorder = {'type' : 'delete', 
            'side' : 'bid', 
          'quantity' : 300, 
          'price' : 2226400,
          'trade_id' : 0,
          'order_id' : 11,#orderbook.orderbook.INITID,
          "timestamp": '34200.1'}
orderbook_2.processOrder(testorder,from_data=True,verbose=False)
print(orderbook_2)

In [ ]:
orderbook.orderbook.INITID

In [ ]:
limit_orders = []
order_id_list = [15000000 + i for i in range(10)]
trade_id = 10086
# timestamp = datetime(34200.000000001)
timestamp = str(34200.000000001)
item = {'type' : 'limit', 
    'side' : 'bid', 
      'quantity' : 100, 
      'price' : 2226400,
      'trade_id' : trade_id,
      'order_id' : order_id_list[0],
      "timestamp": timestamp}

In [ ]:
orderbook_1.orderbook_array[1,0,:,:]

In [ ]:
orderbook_1.orderbook_array[1,0,:,:]

In [ ]:
testorder = {'type' : 'delete', 
            'side' : 'bid', 
          'quantity' : 300, 
          'price' : 2226400,
          'trade_id' : 0,
          'order_id' : 11,#orderbook.orderbook.INITID,
          "timestamp": '34200.1'}
orderbook_2.processOrder(testorder,from_data=True,verbose=False)
print(orderbook_2)

In [ ]:
orderbook.orderbook.INITID

In [386]:
limit_orders = []
order_id_list = [15000000 + i for i in range(10)]
trade_id = 10086
# timestamp = datetime(34200.000000001)
timestamp = str(34200.000000001)
item = {'type' : 'limit', 
    'side' : 'bid', 
      'quantity' : 100, 
      'price' : 2226400,
      'trade_id' : trade_id,
      'order_id' : order_id_list[0],
      "timestamp": timestamp}

Quantity    19  |  Price  2160000  |  Order_ID   26671093  |  Time 35567.228747576


In [383]:
orderbook_1.orderbook_array[1,0,:,:]

Array([[      29,  2160000,        0, 38916890,    36391,  9541395],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       

In [ ]:
testorder = {'type' : 'delete', 
            'side' : 'bid', 
          'quantity' : 300, 
          'price' : 2226400,
          'trade_id' : 0,
          'order_id' : 11,#orderbook.orderbook.INITID,
          "timestamp": '34200.1'}
orderbook_2.processOrder(testorder,from_data=True,verbose=False)
print(orderbook_2)

In [340]:
orderbook.orderbook.INITID

90000000

In [373]:
limit_orders = []
order_id_list = [15000000 + i for i in range(10)]
trade_id = 10086
# timestamp = datetime(34200.000000001)
timestamp = str(34200.000000001)
item = {'type' : 'limit', 
    'side' : 'bid', 
      'quantity' : 100, 
      'price' : 2226400,
      'trade_id' : trade_id,
      'order_id' : order_id_list[0],
      "timestamp": timestamp}

SyntaxError: invalid syntax (<unknown>, line 1)

In [396]:
orderbook_1.process_order(item)

(Array([[      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200]],      dtype=int32),
 Array([       1,        1,      100,  2226400,    10086, 15000000,
           34200,        1], dtype=int32))

In [397]:
orderbook_1.get_L2_state()

Array([2229100,    1507, 2226400,    1500, 2234000,      50, 2222200,
           200, 2237200,     300, 2221500,     100, 2237500,     100,
       2221400,      20, 2240000,     200, 2221000,      14, 2242500,
            20, 2218300,      20, 2247500,      20, 2210000,     100,
       2250000,     100, 2203200,     100, 2252500,      20, 2200000,
            97, 2257500,      40, 2190000,      50, 2262700,       7,
       2185100,       1, 2264900,       1, 2180000,       4, 2270000,
             5, 2153200,     100, 2290000,      70, 2150700,       1,
       2299000,     100, 2133600,     300, 2300000,     139, 2132600,
           100, 2301400,     100, 2100900,      50, 2318300,     100,
       2100000,      50, 2318500,     300, 2096000,      15, 2330000,
            50, 2088200,     100, 2340000,      50, 2078000,     300,
       2344300,       5, 2066900,     100, 2362900,     100, 2050000,
            75, 2364200,      60, 2030000,      10, 2374300,     300,
       2000000,     

In [362]:
item

{'type': 'limit',
 'side': 'bid',
 'quantity': 100,
 'price': 280000,
 'trade_id': 10086,
 'order_id': 15000000,
 'timestamp': '34200.000000001'}

In [412]:
cpu=[[2226400     200 2224900       8]
     [2228900     100 2224500      17]
     [2229100    1507 2224400       2]
     [2230000      26 2224100     265]
     [2233500      23 2224000      42]
     [2233600     100 2221500     100]
     [2234000      50 2221400      20]
     [2235000     390 2221000      14]
     [2237500     100 2218300      20]
     [2240000     250 2210000     100]
     [2241000     100 2203200     100]
     [2242500      20 2200000      97]
     [2247500      20 2190000      50]
     [2250000     100      -1      -1]
     [2252500      20      -1      -1]
     [2257500      40      -1      -1]]



[[2226400      90 2225000     928]
 [2228400     400 2224900       8]
 [2228500     400 2224500      17]
 [2228600     400 2224400       2]
 [2228700     400 2224100     265]
 [2228800     400      -1       0]
 [2228900     900      -1       0]
 [2229000     600      -1       0]
 [2229100    1707      -1       0]
 [2230000      26      -1       0]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1227407754.py, line 1)

##Testing time of different ops

In [8]:
test=jnp.ones([2,5,100,5])

In [9]:
test[0,0,0:3,:]

Array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32)

In [10]:
%timeit jnp.concatenate([test[0,0,0:3,:],jnp.array([[2,2,2,2,2]]),test[0,0,3:-1,:]],axis=0).block_until_ready()

823 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
def old_version(orderlist):
    orderlist=jnp.insert(orderlist,jnp.array(3),jnp.array([[2,2,2,2,2]]),axis=0)
    #Remove last element
    #TODO: if last element is non-null merge with penultimate order (or something else?)
    orderlist=jnp.delete(orderlist,jnp.shape(orderlist)[0]-1,axis=0)
    return orderlist

In [12]:
def new_version(orderlist):
    return jnp.concatenate([orderlist[0:3,:],jnp.array([[2,2,2,2,2]]),orderlist[3:-1,:]],axis=0)

In [13]:
old_version_jitted=jax.jit(old_version)
new_version_jitted=jax.jit(new_version)

In [14]:
%timeit old_version_jitted(test[0,0,:,:]).block_until_ready()

280 µs ± 6.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit new_version_jitted(test[0,0,:,:]).block_until_ready()

277 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
%timeit a,b=jax.block_until_ready(job.processLMTOrder(ordersJNP[horizon+1],orderbook_1.orderbook_array,jnp.ones([5,7])*-1 ))

298 µs ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%timeit a,b=jax.block_until_ready(job.processMKTOrder(ordersJNP[horizon+1],orderbook_1.orderbook_array,jnp.ones([5,7])*-1 ))

335 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%timeit a,b=jax.block_until_ready(job.cancelOrder(ordersJNP[horizon],orderbook_1.orderbook_array,jnp.ones([5,7])*-1 ))

1.04 ms ± 89.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%timeit a,b=jax.block_until_ready(job.delOrder_2arg_old(ordersJNP[horizon],orderbook_1.orderbook_array,jnp.ones([5,7])*-1 ))

1.55 ms ± 141 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%timeit a,b=jax.block_until_ready(job.delOrder_2arg(ordersJNP[horizon],orderbook_1.orderbook_array,jnp.ones([5,7])*-1 ))

707 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
def where_function(orderbook,orderID): #orderbook_1.orderbook_array,38921277):
    idx=jnp.where(orderbook_1.orderbook_array[:,:,:,3]==38921277,size=1,fill_value=-1)
    return idx

wfj=jax.jit(where_function)

%timeit idx=jax.block_until_ready(wfj(orderbook_1.orderbook_array,38921277))

5.84 µs ± 37 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%timeit jax.block_until_ready(job.delOrder_3arg(ordersJNP[horizon+2],orderbook_1.orderbook_array,idx))

NameError: name 'idx' is not defined

In [29]:
ordersJNP[horizon+2]

Array([        3,        -1,       100,   2162100,         0,  38921277,
           36395, 700799735], dtype=int32)

In [30]:
orderbook_1.orderbook_array[0,4,0:3,:]

Array([[      100,   2161800,         0,  38976637,     36395, 377327664],
       [       -1,        -1,        -1,        -1,        -1,        -1],
       [       -1,        -1,        -1,        -1,        -1,        -1]],      dtype=int32)

In [31]:
orderlist=orderbook_1.orderbook_array[0,4,0:3,:]
listLocation=0
ordrelist_2=jnp.concatenate([orderlist[0:listLocation],orderlist[listLocation+1:],jnp.ones((1,6))*-1])
ordrelist_2

Array([[-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.]], dtype=float32)

In [32]:
orderlist

Array([[      100,   2161800,         0,  38976637,     36395, 377327664],
       [       -1,        -1,        -1,        -1,        -1,        -1],
       [       -1,        -1,        -1,        -1,        -1,        -1]],      dtype=int32)

In [33]:
orderlist[0:listLocation].shape

(0, 6)

In [34]:
(jnp.ones((1,6))*-1).shape

(1, 6)

In [35]:
def test_func(listLocation,orderlist):   
    return jnp.concatenate([jnp.roll(jnp.roll(test_list,-listLocation,axis=0)[1:,:],listLocation,axis=0),jnp.ones((1,6))*-1]).astype('int32')

def old_func(listLocation,orderlist):
    orderlist=orderlist.at[listLocation,:].set(-1)

    
    #Extract times - which will be used to sort
    times=orderlist[:,4]
    times_ns=orderlist[:,5]
    #Replace -1 entries with inf time (move to end)
    times=jnp.where(times==-1,jnp.iinfo('int32').max,times)
    times_ns=jnp.where(times_ns==-1,jnp.iinfo('int32').max,times_ns)
    #Get sorted indeces and use to sort orderlist.
    out=jnp.lexsort((times_ns,times),axis=0)
    #out=jnp.argsort(times,axis=0)
    orderlist=orderlist[out,:]

In [36]:
jitted_test_fun=jax.jit(test_func)
jitted_old_fun=jax.jit(old_func)

In [44]:
%timeit test_func(0,orderlist)

314 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [45]:
%timeit jitted_test_fun(0,orderlist)

2.27 µs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [46]:
%timeit jitted_old_fun(0,orderlist)

88.2 µs ± 919 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [38]:
a=lax.dynamic_slice(orderlist, (listLocation+1,0), (1, 6))
a

Array([[-1, -1, -1, -1, -1, -1]], dtype=int32)

In [43]:
listLocation=1
jnp.concatenate([jnp.roll(jnp.roll(test_list,-listLocation,axis=0)[1:,:],listLocation,axis=0),jnp.ones((1,6))*-1]).astype('int32')

Array([[ 1,  1,  1,  1,  1,  1],
       [ 3,  3,  3,  3,  3,  3],
       [-1, -1, -1, -1, -1, -1]], dtype=int32)

In [40]:
test_list=jnp.array([[1,1,1,1,1,1],[2,2,2,2,2,2],[3,3,3,3,3,3]])

In [41]:
jnp.roll(test_list,-listLocation,axis=0)[1:,:]

Array([[3, 3, 3, 3, 3, 3],
       [1, 1, 1, 1, 1, 1]], dtype=int32)

In [42]:
%timeit job.delPrice(orderbook_1.orderbook_array,[0],0,list_loc=0)


83.7 µs ± 663 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
